In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load ground truth data
usgs_raw  = pd.read_csv("USGS_FD.csv", sep=',')
usgs_raw

In [ ]:
# Load ROI data
camera_raw = pd.read_csv("firstdamsite.csv", sep=',')
camera_raw

In [ ]:
# Step 2: Clean USGS data (first row is header info)
usgs_raw.columns = usgs_raw.iloc[0]
usgs_cleaned = usgs_raw.iloc[1:].copy()
usgs_cleaned.columns = usgs_raw.columns

# Step 3: Rename relevant columns
usgs_cleaned = usgs_cleaned.rename(columns={
    '20d': 'datetime',
    '14n': 'stage'
})

# Step 4: Convert datetime and stage to proper formats
usgs_cleaned["datetime"] = pd.to_datetime(usgs_cleaned["datetime"], errors="coerce")
usgs_cleaned["stage"] = pd.to_numeric(usgs_cleaned["stage"], errors="coerce")
usgs_cleaned = usgs_cleaned.dropna(subset=["datetime"])

In [ ]:
# Step 5: Prepare camera dataset timestamps
camera_raw["image_timestamp"] = pd.to_datetime(camera_raw["image_timestamp"], errors="coerce")
camera_raw["RoundedDateTime"] = camera_raw["image_timestamp"].dt.round("15min")

# Step 6: Select relevant USGS data and merge with camera data
usgs_selected = usgs_cleaned[["datetime", "stage"]]
merged_data = camera_raw.merge(usgs_selected, left_on="RoundedDateTime", right_on="datetime", how="inner")
merged_data = merged_data.drop(columns=["RoundedDateTime"])

# Step 7: Filter required fields
filtered_data = merged_data[["image_timestamp", "roi", "iou_score", "stage"]].copy()

In [ ]:
# Step 8: Extract ROI1 and ROI2 from 'roi' string
def split_roi(roi_str):
    try:
        values = list(map(int, roi_str.strip('{}').split(',')))
        return pd.Series(values[:2], index=["ROI1", "ROI2"])
    except:
        return pd.Series([None, None], index=["ROI1", "ROI2"])

roi_split = filtered_data["roi"].apply(split_roi)
final_data = pd.concat([filtered_data.drop(columns=["roi"]), roi_split], axis=1)
final_data = final_data[["image_timestamp", "ROI1", "ROI2", "iou_score", "stage"]]
final_data["stage"] = final_data["stage"] * 30.48

In [ ]:
# Display the updated DataFrame
print(final_data.head())

In [ ]:
final_data = final_data.sort_values("image_timestamp")
# Plot with secondary y-axis for stage data
fig, ax1 = plt.subplots(figsize=(14, 6))

# Primary y-axis for ROI1 and ROI2
ax1.plot(final_data["image_timestamp"], final_data["ROI1"], label="ROI1 Pixels", color='tab:blue')
ax1.plot(final_data["image_timestamp"], final_data["ROI2"], label="ROI2 Pixels", color='tab:green')
ax1.set_xlabel("Image Timestamp")
ax1.set_ylabel("ROI Pixel Values", color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.legend(loc="upper left")

# Secondary y-axis for stage data
ax2 = ax1.twinx()
ax2.plot(final_data["image_timestamp"], final_data["stage"], label="USGS Stage (cm)", color='tab:red', linestyle='--')
ax2.set_ylabel("Stage (cm)", color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.legend(loc="upper right")

# Final plot adjustments
plt.title("Time Series of ROI Pixel Values and USGS Stage")
fig.autofmt_xdate()
fig.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
# Plot IoU score distribution to assess threshold
plt.figure(figsize=(10, 5))
plt.hist(final_data["iou_score"], bins=30, edgecolor='black', color='skyblue')
plt.title("Distribution of IoU Scores")
plt.xlabel("IoU Score")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Filter the data to keep only rows with IoU >= 0.7
filtered_high_iou = final_data[final_data["iou_score"] >= 0.7].copy()

filtered_high_iou.head()

In [ ]:
# Filter the main high-IoU dataset to include only images captured between 07:00 and 20:00
time_filtered_df = filtered_high_iou.copy()
time_filtered_df["hour"] = time_filtered_df["image_timestamp"].dt.hour

# Include only records between 7 AM and 8 PM (inclusive)
time_filtered_df = time_filtered_df[(time_filtered_df["hour"] >= 7) & (time_filtered_df["hour"] <= 20)].copy()

# Drop the 'hour' column now that filtering is done
time_filtered_df.drop(columns=["hour"], inplace=True)

In [ ]:
time_filtered_df

In [ ]:
time_filtered_df = time_filtered_df.sort_values("image_timestamp")

# Re-plot with stage on a secondary y-axis and ROI values on primary y-axis

fig, ax1 = plt.subplots(figsize=(14, 6))

# Primary y-axis for ROI1 and ROI2
ax1.plot(time_filtered_df["image_timestamp"], time_filtered_df["ROI1"], label="ROI1 Pixels", color='tab:blue', alpha=0.6)
ax1.plot(time_filtered_df["image_timestamp"], time_filtered_df["ROI2"], label="ROI2 Pixels", color='tab:green', alpha=0.6)
ax1.set_xlabel("Image Timestamp")
ax1.set_ylabel("ROI Pixel Values", color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.legend(loc="upper left")

# Secondary y-axis for USGS stage
ax2 = ax1.twinx()
ax2.plot(time_filtered_df["image_timestamp"], time_filtered_df["stage"], label="USGS Stage (cm)", color='tab:red', linestyle='--', alpha=0.9)
ax2.set_ylabel("Stage (cm)", color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.legend(loc="upper right")

plt.title("Time Series with Dual Axes: ROI Pixels vs USGS Stage (7AM–8PM, IoU ≥ 0.7)")
plt.grid(True)
fig.autofmt_xdate()
fig.tight_layout()
plt.show()


In [ ]:
# Step 1: Remove rows where ROI1 or ROI2 <= 100 (likely invalid segmentations)
cleaned_df = time_filtered_df[(time_filtered_df["ROI1"] > 100) & (time_filtered_df["ROI2"] > 100)].copy()

# Step 2: Clip ROI1 and ROI2 at their 99th percentile
roi1_clip = cleaned_df["ROI1"].quantile(0.99)
roi2_clip = cleaned_df["ROI2"].quantile(0.99)

cleaned_df["ROI1"] = cleaned_df["ROI1"].clip(upper=roi1_clip)
cleaned_df["ROI2"] = cleaned_df["ROI2"].clip(upper=roi2_clip)

In [ ]:
# Sort the cleaned DataFrame for plotting
cleaned_df_sorted = cleaned_df.sort_values("image_timestamp")

# Plot time series with dual y-axes
fig, ax1 = plt.subplots(figsize=(14, 6))

# Left y-axis for ROI1 and ROI2
ax1.plot(cleaned_df_sorted["image_timestamp"], cleaned_df_sorted["ROI1"], label="ROI1 Pixels", color='tab:blue', alpha=0.7)
ax1.plot(cleaned_df_sorted["image_timestamp"], cleaned_df_sorted["ROI2"], label="ROI2 Pixels", color='tab:green', alpha=0.7)
ax1.set_xlabel("Image Timestamp")
ax1.set_ylabel("ROI Pixel Values", color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.legend(loc="upper left")

# Right y-axis for stage
ax2 = ax1.twinx()
ax2.plot(cleaned_df_sorted["image_timestamp"], cleaned_df_sorted["stage"], label="USGS Stage (cm)", color='tab:red', linestyle='--', alpha=0.9)
ax2.set_ylabel("Stage (cm)", color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.legend(loc="upper right")

plt.title("Cleaned Time Series: ROI Pixels vs Stage (7AM–8PM, IoU ≥ 0.7, Outliers Removed)")
plt.grid(True)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Apply the cutoff date to remove unreliable early segmentation data
final_df = cleaned_df[cleaned_df["image_timestamp"] >= "2024-08-01"].copy()

# Compute Pearson and Spearman correlations on the final dataset
pearson_corr_final = final_df[["ROI1", "ROI2", "stage"]].corr(method="pearson")
spearman_corr_final = final_df[["ROI1", "ROI2", "stage"]].corr(method="spearman")

pearson_corr_final, spearman_corr_final

In [ ]:
# Function to apply IQR-based outlier removal
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply IQR filtering to both ROI1 and ROI2
iqr_filtered_df = final_df.copy()
iqr_filtered_df = remove_outliers_iqr(iqr_filtered_df, "ROI1")
iqr_filtered_df = remove_outliers_iqr(iqr_filtered_df, "ROI2")

# Sort for plotting
iqr_filtered_df_sorted = iqr_filtered_df.sort_values("image_timestamp")

# Plot the new time series after IQR filtering
fig, ax1 = plt.subplots(figsize=(14, 6))

# ROI1 and ROI2 on left y-axis
ax1.plot(iqr_filtered_df_sorted["image_timestamp"], iqr_filtered_df_sorted["ROI1"], label="ROI1 Pixels", color='tab:blue', alpha=0.7)
ax1.plot(iqr_filtered_df_sorted["image_timestamp"], iqr_filtered_df_sorted["ROI2"], label="ROI2 Pixels", color='tab:green', alpha=0.7)
ax1.set_xlabel("Image Timestamp", fontsize=14)
ax1.set_ylabel("ROI Pixel Values", color='black', fontsize=14)
ax1.tick_params(axis='y', labelcolor='black', labelsize=12)
ax1.tick_params(axis='x', labelsize=12)
ax1.legend(loc="upper left", fontsize=12)

# Stage on right y-axis
ax2 = ax1.twinx()
ax2.plot(iqr_filtered_df_sorted["image_timestamp"], iqr_filtered_df_sorted["stage"], label="USGS Stage (cm)", color='tab:red', linestyle='--', alpha=0.9)
ax2.set_ylabel("Stage (cm)", color='tab:red', fontsize=14)
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=12)
ax2.legend(loc="upper right", fontsize=12)

plt.title("Time Series After IQR-Based Outlier Removal (ROI1 & ROI2)", fontsize=16)
plt.grid(True)
plt.tight_layout()
plt.savefig("timeseries_stage.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
iqr_filtered_df_sorted.to_csv("iqr_filtered_df_sorted.csv", index=False)

In [ ]:
iqr_filtered_df_sorted

In [ ]:
# Compute Pearson and Spearman correlations on the IQR-filtered final dataset
pearson_corr_iqr = iqr_filtered_df[["ROI1", "ROI2", "stage"]].corr(method="pearson")
spearman_corr_iqr = iqr_filtered_df[["ROI1", "ROI2", "stage"]].corr(method="spearman")

pearson_corr_iqr, spearman_corr_iqr

In [ ]:
import seaborn as sns
# Plot side-by-side heatmaps
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pearson
sns.heatmap(
    pearson_corr_iqr,
    annot=True,
    cmap="coolwarm",
    vmin=-1, vmax=1,
    ax=axes[0],
    annot_kws={"size": 12},  # annotation font size
    cbar_kws={"shrink": 0.8}
)
axes[0].set_title("Pearson Correlation", fontsize=14)
axes[0].tick_params(axis='both', labelsize=12)

# Spearman
sns.heatmap(
    spearman_corr_iqr,
    annot=True,
    cmap="coolwarm",
    vmin=-1, vmax=1,
    ax=axes[1],
    annot_kws={"size": 12},
    cbar_kws={"shrink": 0.8}
)
axes[1].set_title("Spearman Correlation", fontsize=14)
axes[1].tick_params(axis='both', labelsize=12)

plt.tight_layout()

# Save the figure
plt.savefig("correlation_stage.png", dpi=300, bbox_inches='tight')  # save as PNG
# plt.savefig("correlation_heatmaps.pdf")  # alternative for PDF

plt.show()

In [ ]:
# Perform extended EDA on the final IQR-filtered dataset

eda_summary_final = {
    "Total Entries": len(iqr_filtered_df),
    "ROI1 Range": (iqr_filtered_df["ROI1"].min(), iqr_filtered_df["ROI1"].max()),
    "ROI2 Range": (iqr_filtered_df["ROI2"].min(), iqr_filtered_df["ROI2"].max()),
    "Stage Range": (iqr_filtered_df["stage"].min(), iqr_filtered_df["stage"].max()),
    "Mean ROI1": iqr_filtered_df["ROI1"].mean(),
    "Mean ROI2": iqr_filtered_df["ROI2"].mean(),
    "Mean Stage": iqr_filtered_df["stage"].mean(),
    "STD ROI1": iqr_filtered_df["ROI1"].std(),
    "STD ROI2": iqr_filtered_df["ROI2"].std(),
    "STD Stage": iqr_filtered_df["stage"].std(),
    "Skewness ROI1": iqr_filtered_df["ROI1"].skew(),
    "Skewness ROI2": iqr_filtered_df["ROI2"].skew(),
    "Skewness Stage": iqr_filtered_df["stage"].skew(),
    "Kurtosis ROI1": iqr_filtered_df["ROI1"].kurtosis(),
    "Kurtosis ROI2": iqr_filtered_df["ROI2"].kurtosis(),
    "Kurtosis Stage": iqr_filtered_df["stage"].kurtosis()
}

# Print summary if desired
for k, v in eda_summary_final.items():
    print(f"{k}: {v}")

# Check pairwise scatterplot matrix
# Pairplot with font scaling and transparency
pairplot = sns.pairplot(
    iqr_filtered_df[["ROI1", "ROI2", "stage"]],
    corner=True,
    plot_kws={"alpha": 0.4, "s": 25},  # scatter dot size and transparency
    height=2.5,
)
# Set title with larger font
pairplot.fig.suptitle("Pairwise Feature Distributions and Relationships", fontsize=14, y=1.03)

# Update font size for all axes in the pairplot
for ax in pairplot.axes.flat:
    if ax is not None:
        ax.tick_params(axis='both', labelsize=12)
        ax.xaxis.label.set_size(13)
        ax.yaxis.label.set_size(13)

# Save figure
pairplot.savefig("pairwise_distribution.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# === Step 1: Prepare features and target ===
X = iqr_filtered_df[["ROI1", "ROI2"]]
y = iqr_filtered_df["stage"]
timestamps = iqr_filtered_df["image_timestamp"]

# === Step 2: Train-Test Split (80/20) ===
X_train, X_test, y_train, y_test, ts_train, ts_test = train_test_split(
    X, y, timestamps, test_size=0.2, random_state=42
)

# === Step 3: Fit model on training data ===
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# === Step 4: Predict on train and test sets ===
y_train_pred = lin_reg.predict(X_train)
y_test_pred = lin_reg.predict(X_test)

# === Step 5: Compute train-test metrics ===
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# === Step 6: 5-Fold Cross-Validation (on full data) ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# R²
r2_scores = cross_val_score(lin_reg, X, y, cv=kf, scoring="r2")

# RMSE (neg MSE → RMSE)
neg_mse_scores = cross_val_score(lin_reg, X, y, cv=kf, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-neg_mse_scores)

# === Step 7: Summarize results ===
results = {
    "Train R²": train_r2,
    "Test R²": test_r2,
    "Train RMSE": train_rmse,
    "Test RMSE": test_rmse,
    "Train MAE": train_mae,
    "Test MAE": test_mae,
    "CV R² Mean": np.mean(r2_scores),
    "CV R² Std": np.std(r2_scores),
    "CV RMSE Mean": np.mean(rmse_scores),
    "CV RMSE Std": np.std(rmse_scores),
}

# === Step 8: Display nicely ===
for k, v in results.items():
    print(f"{k}: {v:.4f}")


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# === Step 1: Polynomial Feature Expansion (Degree 2) ===
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
X_full_poly = poly.transform(X)  # for cross-validation

# === Optional: View feature names ===
# print(poly.get_feature_names_out(["ROI1", "ROI2"]))

# === Step 2: Train Model ===
model = LinearRegression()
model.fit(X_train_poly, y_train)

# === Step 3: Predictions ===
y_train_pred = model.predict(X_train_poly)
y_test_pred = model.predict(X_test_poly)

# === Step 4: Train-Test Metrics ===
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# === Step 5: Cross-Validation ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(model, X_full_poly, y, cv=kf, scoring="r2")
cv_neg_mse = cross_val_score(model, X_full_poly, y, cv=kf, scoring="neg_mean_squared_error")
cv_rmse = np.sqrt(-cv_neg_mse)

# === Step 6: Combine Metrics ===
metrics_poly = {
    "Train R²": train_r2,
    "Test R²": test_r2,
    "Train RMSE": train_rmse,
    "Test RMSE": test_rmse,
    "Train MAE": train_mae,
    "Test MAE": test_mae,
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse),
}

# === Step 7: Print Results Nicely ===
for k, v in metrics_poly.items():
    print(f"{k}: {v:.4f}")

In [ ]:
# Define global axis limits with margin
buffer = 0.05 * (y.max() - y.min())
min_val = min(y.min(), y_train_pred.min(), y_test_pred.min()) - buffer
max_val = max(y.max(), y_train_pred.max(), y_test_pred.max()) + buffer

# Fit regression lines manually
train_slope, train_intercept = np.polyfit(y_train, y_train_pred, 1)
test_slope, test_intercept = np.polyfit(y_test, y_test_pred, 1)
x_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(14, 6))

# Train set
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, alpha=0.6, color='dodgerblue', edgecolor='k', s=50, label='Predictions')
plt.plot(x_line, x_line, 'r--', linewidth=1.5, label='1:1 Line')
plt.plot(x_line, train_slope * x_line + train_intercept, 'b-', linewidth=2, label='Regression Line')
plt.title("Train Set: Actual vs Predicted Stage", fontsize=14)
plt.xlabel("Actual Stage (cm)", fontsize=12)
plt.ylabel("Predicted Stage (cm)", fontsize=12)
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)
plt.grid(True)
plt.legend(fontsize=11)
plt.tick_params(labelsize=11)

# Add regression equation as boxed text
plt.text(
    0.05, 0.95,
    f"$y = {train_slope:.2f}x + {train_intercept:.2f}$",
    transform=plt.gca().transAxes,
    fontsize=11,
    verticalalignment='top',
    bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.4')
)

# Test set
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred, alpha=0.6, color='forestgreen', edgecolor='k', s=50, label='Predictions')
plt.plot(x_line, x_line, 'r--', linewidth=1.5, label='1:1 Line')
plt.plot(x_line, test_slope * x_line + test_intercept, 'g-', linewidth=2, label='Regression Line')
plt.title("Test Set: Actual vs Predicted Stage", fontsize=14)
plt.xlabel("Actual Stage (cm)", fontsize=12)
plt.ylabel("Predicted Stage (cm)", fontsize=12)
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)
plt.grid(True)
plt.legend(fontsize=11)
plt.tick_params(labelsize=11)

# Add regression equation as boxed text
plt.text(
    0.05, 0.95,
    f"$y = {test_slope:.2f}x + {test_intercept:.2f}$",
    transform=plt.gca().transAxes,
    fontsize=11,
    verticalalignment='top',
    bbox=dict(facecolor='white', edgecolor='green', boxstyle='round,pad=0.4')
)

plt.tight_layout()

# Save the figure
plt.savefig("scatter_stage.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
slope, intercept = np.polyfit(y_test, y_test_pred, 1)
print(f"Test slope: {slope:.3f}, intercept: {intercept:.3f}")


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# === Step 3: Tuned Random Forest Regressor ===
rf_tuned = RandomForestRegressor(
    n_estimators=100,
    max_depth=6,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)
rf_tuned.fit(X_train, y_train)

# === Step 4: Predictions ===
y_train_pred = rf_tuned.predict(X_train)
y_test_pred = rf_tuned.predict(X_test)

# === Step 5: Evaluation ===
metrics_rf = {
    "Train R²": r2_score(y_train, y_train_pred),
    "Test R²": r2_score(y_test, y_test_pred),
    "Train MAE": mean_absolute_error(y_train, y_train_pred),
    "Test MAE": mean_absolute_error(y_test, y_test_pred),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred)),
    "Train MSE": mean_squared_error(y_train, y_train_pred),
    "Test MSE": mean_squared_error(y_test, y_test_pred),
}

# === Step 6: Optional Cross-Validation (on full set) ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(rf_tuned, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(rf_tuned, X, y, cv=cv, scoring="neg_mean_squared_error"))

metrics_rf.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse),
})

# === Step 7: Display nicely ===
for k, v in metrics_rf.items():
    print(f"{k}: {v:.4f}")

In [ ]:
import xgboost as xgb

# === Step 1: Define XGBoost Regressor ===
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='reg:squarederror',  # prevents warning for regression
    random_state=42
)

# === Step 2: Train on Train Set ===
xgb_model.fit(X_train, y_train)

# === Step 3: Predict ===
y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

# === Step 4: Train-Test Evaluation ===
xgb_metrics = {
    "Train R²": r2_score(y_train, y_train_pred_xgb),
    "Test R²": r2_score(y_test, y_test_pred_xgb),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_xgb),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_xgb),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_xgb)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_xgb)),
    "Train MSE": mean_squared_error(y_train, y_train_pred_xgb),
    "Test MSE": mean_squared_error(y_test, y_test_pred_xgb)
}

# === Step 5: Cross-Validation ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(xgb_model, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(xgb_model, X, y, cv=cv, scoring="neg_mean_squared_error"))

xgb_metrics.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse)
})

# === Step 6: Print Clean Summary ===
for k, v in xgb_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
import lightgbm as lgb

# === Step 1: Define LightGBM Regressor ===
lgb_model = lgb.LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    num_leaves=31,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

# === Step 2: Train ===
lgb_model.fit(X_train, y_train)

# === Step 3: Predict ===
y_train_pred_lgb = lgb_model.predict(X_train)
y_test_pred_lgb = lgb_model.predict(X_test)

# === Step 4: Evaluate on Train and Test ===
lgb_metrics = {
    "Train R²": r2_score(y_train, y_train_pred_lgb),
    "Test R²": r2_score(y_test, y_test_pred_lgb),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_lgb),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_lgb),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_lgb)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_lgb)),
    "Train MSE": mean_squared_error(y_train, y_train_pred_lgb),
    "Test MSE": mean_squared_error(y_test, y_test_pred_lgb)
}

# === Step 5: Cross-Validation ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(lgb_model, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(lgb_model, X, y, cv=cv, scoring="neg_mean_squared_error"))

lgb_metrics.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse)
})

# === Step 6: Print Results ===
for k, v in lgb_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# === Step 2: Define SVR Pipeline ===
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.01)
)

# === Step 3: Train and Predict ===
svr_model.fit(X_train, y_train)
y_train_pred_svr = svr_model.predict(X_train)
y_test_pred_svr = svr_model.predict(X_test)

# === Step 4: Evaluation Metrics ===
svr_metrics = {
    "Train R²": r2_score(y_train, y_train_pred_svr),
    "Test R²": r2_score(y_test, y_test_pred_svr),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_svr),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_svr),
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_svr)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_svr)),
    "Train MSE": mean_squared_error(y_train, y_train_pred_svr),
    "Test MSE": mean_squared_error(y_test, y_test_pred_svr)
}

# === Step 5: Cross-Validation ===
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2 = cross_val_score(svr_model, X, y, cv=cv, scoring="r2")
cv_rmse = np.sqrt(-cross_val_score(svr_model, X, y, cv=cv, scoring="neg_mean_squared_error"))

svr_metrics.update({
    "CV R² Mean": np.mean(cv_r2),
    "CV R² Std": np.std(cv_r2),
    "CV RMSE Mean": np.mean(cv_rmse),
    "CV RMSE Std": np.std(cv_rmse)
})

# === Step 6: Print Results ===
for k, v in svr_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
# Step 1: Prepare the data
df_sorted = iqr_filtered_df.sort_values("image_timestamp").reset_index(drop=True)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

df_scaled = df_sorted.copy()
df_scaled[["ROI1", "ROI2"]] = scaler_X.fit_transform(df_sorted[["ROI1", "ROI2"]])
df_scaled["stage"] = scaler_y.fit_transform(df_sorted[["stage"]])


In [ ]:
# Step 2: Dataset Class
class StageSequenceDataset(Dataset):
    def __init__(self, df, seq_len=10):
        self.seq_len = seq_len
        self.features = df[["ROI1", "ROI2"]].values.astype(np.float32)
        self.targets = df["stage"].values.astype(np.float32)

    def __len__(self):
        return len(self.features) - self.seq_len

    def __getitem__(self, idx):
        X = self.features[idx:idx + self.seq_len]
        y = self.targets[idx + self.seq_len]
        return torch.tensor(X), torch.tensor(y)

In [ ]:
# Step 3: LSTM Model
class StageLSTM(nn.Module):
    def __init__(self, input_size=2, hidden_size=128, num_layers=2, dropout=0.2):
        super(StageLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out.squeeze()

In [ ]:
# Define dataset
seq_len = 10
dataset = StageSequenceDataset(df_scaled, seq_len=seq_len)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * train_size)

actual_train_ds = Subset(dataset, range(train_size - val_size))
val_ds = Subset(dataset, range(train_size - val_size, train_size))
test_ds = Subset(dataset, range(train_size, len(dataset)))

train_loader = DataLoader(actual_train_ds, batch_size=32, shuffle=False)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

# Initialize model
model = StageLSTM()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping variables
best_val_loss = float('inf')
patience = 5
patience_counter = 0
best_model_state = None

# Training loop with early stopping
epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            preds = model(X_batch)
            val_loss += criterion(preds, y_batch).item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

    # Early stopping condition
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

# Load best model
if best_model_state:
    model.load_state_dict(best_model_state)


In [ ]:
# Evaluation on test set
model.eval()
y_pred, y_true = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch)
        y_pred.extend(preds.numpy())
        y_true.extend(y_batch.numpy())

# Inverse scaling
y_pred_inv = scaler_y.inverse_transform(np.array(y_pred).reshape(-1, 1)).flatten()
y_true_inv = scaler_y.inverse_transform(np.array(y_true).reshape(-1, 1)).flatten()

print("LSTM with Early Stopping (Test)")
print("Test R²:", r2_score(y_true_inv, y_pred_inv))
print("Test MAE:", mean_absolute_error(y_true_inv, y_pred_inv))

# Evaluation on training set
y_train_pred, y_train_true = [], []
with torch.no_grad():
    for X_batch, y_batch in DataLoader(actual_train_ds, batch_size=32, shuffle=False):
        preds = model(X_batch)
        y_train_pred.extend(preds.numpy())
        y_train_true.extend(y_batch.numpy())

# Inverse scaling
y_train_pred_inv = scaler_y.inverse_transform(np.array(y_train_pred).reshape(-1, 1)).flatten()
y_train_true_inv = scaler_y.inverse_transform(np.array(y_train_true).reshape(-1, 1)).flatten()

print("LSTM with Early Stopping (Train)")
print("Train R²:", r2_score(y_train_true_inv, y_train_pred_inv))
print("Train MAE:", mean_absolute_error(y_train_true_inv, y_train_pred_inv))


In [ ]:
# === Step 1: Combine all model metrics into a dictionary ===
model_results = {
    "SVR": svr_metrics,
    "Random Forest": metrics_rf,
    "XGBoost": xgb_metrics,
    "LightGBM": lgb_metrics,
    "Polynomial Regression (d=2)": metrics_poly,
    "Linear Regression": results
}

# === Step 2: Create DataFrame ===
ranking_df = pd.DataFrame(model_results).T[[
    "Test R²", "Test MAE", "Test RMSE", "CV R² Mean", "CV RMSE Mean"
]]
ranking_df.columns = ["R²", "MAE", "RMSE", "CV_R²", "CV_RMSE"]

# === Step 3: Rank models (lower rank is better) ===
ranking_df["R²_rank"] = ranking_df["R²"].rank(ascending=False)
ranking_df["MAE_rank"] = ranking_df["MAE"].rank(ascending=True)
ranking_df["RMSE_rank"] = ranking_df["RMSE"].rank(ascending=True)
ranking_df["CV_R²_rank"] = ranking_df["CV_R²"].rank(ascending=False)
ranking_df["CV_RMSE_rank"] = ranking_df["CV_RMSE"].rank(ascending=True)

# === Step 4: Calculate average rank and sort ===
ranking_df["Avg_Rank"] = ranking_df[
    ["R²_rank", "MAE_rank", "RMSE_rank", "CV_R²_rank", "CV_RMSE_rank"]
].mean(axis=1)

ranking_df = ranking_df.sort_values("Avg_Rank")

# Display or save
print(ranking_df[["R²", "MAE", "RMSE", "CV_R²", "CV_RMSE", "Avg_Rank"]])
# ranking_df.to_csv("model_ranking.csv", index=True)

In [ ]:
# === STEP 1: Predictions from the top 3 models ===
# Replace these with your actual prediction variables
pred_svr = y_test_pred_svr      # SVR prediction
pred_rf = y_test_pred           # RF prediction
pred_xgb = y_test_pred_xgb      # XGBoost prediction

# === STEP 2: Create stacked combinations ===
stack_svr_rf   = np.vstack([pred_svr, pred_rf]).T
stack_svr_xgb  = np.vstack([pred_svr, pred_xgb]).T
stack_rf_xgb   = np.vstack([pred_rf, pred_xgb]).T
stack_all_3    = np.vstack([pred_svr, pred_rf, pred_xgb]).T

# === STEP 3: Meta-model (stacked ensemble) predictions ===
stacked_svr_rf   = LinearRegression().fit(stack_svr_rf, y_test).predict(stack_svr_rf)
stacked_svr_xgb  = LinearRegression().fit(stack_svr_xgb, y_test).predict(stack_svr_xgb)
stacked_rf_xgb   = LinearRegression().fit(stack_rf_xgb, y_test).predict(stack_rf_xgb)
stacked_all_3    = LinearRegression().fit(stack_all_3, y_test).predict(stack_all_3)

# === STEP 4: Simple averages ===
avg_svr_rf   = np.mean(stack_svr_rf, axis=1)
avg_svr_xgb  = np.mean(stack_svr_xgb, axis=1)
avg_rf_xgb   = np.mean(stack_rf_xgb, axis=1)
avg_all_3    = np.mean(stack_all_3, axis=1)

# === STEP 5: Weighted average (customizable weights) ===
weighted_avg = 0.4 * pred_rf + 0.4 * pred_svr + 0.2 * pred_xgb

# === STEP 6: Evaluation function ===
def evaluate(y_true, y_pred):
    return {
        "R²": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred))
    }

# === STEP 7: Compute metrics for all ensemble strategies ===
ensemble_results = {
    "Simple Avg (SVR + RF)": evaluate(y_test, avg_svr_rf),
    "Simple Avg (SVR + XGB)": evaluate(y_test, avg_svr_xgb),
    "Simple Avg (RF + XGB)": evaluate(y_test, avg_rf_xgb),
    "Simple Avg (All 3)": evaluate(y_test, avg_all_3),
    "Weighted Avg (0.4 RF + 0.4 SVR + 0.2 XGB)": evaluate(y_test, weighted_avg),
    "Stacked (SVR + RF)": evaluate(y_test, stacked_svr_rf),
    "Stacked (SVR + XGB)": evaluate(y_test, stacked_svr_xgb),
    "Stacked (RF + XGB)": evaluate(y_test, stacked_rf_xgb),
    "Stacked (All 3)": evaluate(y_test, stacked_all_3)
}

# === STEP 8: Convert to DataFrame and sort ===
ensemble_df = pd.DataFrame(ensemble_results).T
ensemble_df = ensemble_df.sort_values("R²", ascending=False)

# Display the final table
print(ensemble_df)
# Optional: Save to CSV
# ensemble_df.to_csv("ensemble_comparison_top3.csv", index=True)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# === BASE MODELS ===
base_models = {
    "SVR": svr_model,
    "Random Forest": rf_tuned,
    "XGBoost": xgb_model
}

# === PREPARE OOF PREDICTIONS ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof_preds = {name: np.zeros_like(y_train) for name in base_models}
test_preds = {name: [] for name in base_models}

# === 1. Generate OOF for training and test predictions ===
for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    for name, model in base_models.items():
        model.fit(X_tr, y_tr)
        oof_preds[name][val_idx] = model.predict(X_val)
        test_preds[name].append(model.predict(X_test))  # accumulate fold predictions

# === 2. Stack into new training and test matrices ===
X_meta_train = np.vstack([oof_preds[name] for name in base_models]).T
X_meta_test = np.mean([np.vstack(test_preds[name]) for name in base_models], axis=1).T

# === 3. Train meta-model on OOF predictions ===
meta_model = LinearRegression()
meta_model.fit(X_meta_train, y_train)

# === 4. Predict on holdout test set ===
meta_pred_test = meta_model.predict(X_meta_test)

# === 5. Evaluate final stacked model ===
def evaluate(y_true, y_pred):
    return {
        "R²": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred))
    }

stacked_cv_results = evaluate(y_test, meta_pred_test)
print("Cross-Validated Stacked Ensemble Performance:")
for k, v in stacked_cv_results.items():
    print(f"{k}: {v:.4f}")


In [ ]:
# === Set global font to bold ===
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

# === Prepare plot data ===
df_plot = pd.DataFrame({
    "timestamp": ts_test,
    "True Stage": y_test,
    "Simple Avg (SVR + RF)": avg_svr_rf,
    "Weighted Avg (0.4 RF + 0.4 SVR + 0.2 XGB)": weighted_avg,
    "Stacked Ensemble (All 3)": stacked_all_3
}).sort_values("timestamp")

# === Create subplots ===
fig, axs = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# Plot 1: Simple Average
axs[0].plot(df_plot["timestamp"], df_plot["True Stage"], label="True Stage", color="black", linewidth=2)
axs[0].plot(df_plot["timestamp"], df_plot["Simple Avg (SVR + RF)"], label="Simple Avg (SVR + RF)", color="royalblue", linewidth=2)
axs[0].set_title("Simple Average (SVR + RF) vs. True Stage")
axs[0].set_ylabel("Stage (cm)", fontsize=14)
axs[0].legend()
axs[0].tick_params(axis='both', labelsize=14)
axs[0].grid(True)

# Plot 2: Weighted Average
axs[1].plot(df_plot["timestamp"], df_plot["True Stage"], label="True Stage", color="black", linewidth=2)
axs[1].plot(df_plot["timestamp"], df_plot["Weighted Avg (0.4 RF + 0.4 SVR + 0.2 XGB)"], label="Weighted Avg", color="darkorange", linewidth=2)
axs[1].set_title("Weighted Average vs. True Stage")
axs[1].set_ylabel("Stage (cm)", fontsize=14)
axs[1].legend()
axs[1].tick_params(axis='both', labelsize=14)
axs[1].grid(True)

# Plot 3: Stacked Ensemble
axs[2].plot(df_plot["timestamp"], df_plot["True Stage"], label="True Stage", color="black", linewidth=2)
axs[2].plot(df_plot["timestamp"], df_plot["Stacked Ensemble (All 3)"], label="Stacked Ensemble (All 3)", color="seagreen", linewidth=2)
axs[2].set_title("Stacked Ensemble (All 3) vs. True Stage")
axs[2].set_xlabel("Timestamp", fontsize=14)
axs[2].set_ylabel("Stage (cm)", fontsize=14)
axs[2].legend()
axs[2].tick_params(labelsize=14)
axs[2].tick_params(axis='both', labelsize=14)
axs[2].grid(True)

plt.tight_layout()

# === Save figure if needed ===
plt.savefig("stacked_stage.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Titles and predictions
titles = ["Simple Avg (SVR + RF)", "Weighted Avg", "Stacked Ensemble (All 3)"]
predictions = [avg_svr_rf, weighted_avg, stacked_all_3]

# Plot each ensemble prediction vs. actual stage
for ax, name, pred in zip(axs, titles, predictions):
    # Scatter plot
    ax.scatter(y_test, pred, alpha=0.7, edgecolors='k')

    # 1:1 reference line
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="1:1 Line")

    # Regression line
    coeffs = np.polyfit(y_test, pred, deg=1)
    reg_line = np.poly1d(coeffs)
    ax.plot(y_test, reg_line(y_test), color='blue', linewidth=2, label="Regression Line")

    # Labels and formatting
    ax.set_xlabel("Actual Stage (cm)", fontsize=14)
    ax.set_title(name, fontsize=15)
    ax.tick_params(axis='both', labelsize=12)
    ax.legend()

# Shared Y-axis label
axs[0].set_ylabel("Predicted Stage (cm)", fontsize=14)

# Main title
fig.suptitle("Actual vs Predicted Stage (Ensemble Models)", fontsize=16)

# Layout adjustment and save
plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.savefig("ensemble_scatter.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# === Calculate Residuals ===
resid_simple = y_test - avg_svr_rf
resid_weighted = y_test - weighted_avg
resid_stacked = y_test - stacked_all_3

# === Create subplots ===
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Titles and predictions
titles = ["Simple Avg (SVR + RF)", "Weighted Avg", "Stacked Ensemble (All 3)"]
predictions = [avg_svr_rf, weighted_avg, stacked_all_3]
residuals = [resid_simple, resid_weighted, resid_stacked]

# Plot residuals
for ax, name, pred, resid in zip(axs, titles, predictions, residuals):
    ax.scatter(pred, resid, alpha=0.7, edgecolors='k')
    ax.axhline(0, color='red', linestyle='--')
    ax.set_xlabel("Predicted Stage (cm)", fontsize=14)
    ax.set_title(f"{name}\n$R^2$ = {r2_score(y_test, pred):.3f}, MAE = {mean_absolute_error(y_test, pred):.4f}", fontsize=13)
    ax.tick_params(axis='both', labelsize=12)

# Shared Y-axis label
axs[0].set_ylabel("Residual (Actual - Predicted) (cm)", fontsize=14)

# Layout and save
plt.tight_layout()
plt.savefig("ensemble_residuals.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# === Calculate Prediction Errors ===
error_simple = avg_svr_rf - y_test
error_weighted = weighted_avg - y_test
error_stacked = stacked_all_3 - y_test

# === Create histograms of prediction errors ===
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Titles and errors
titles = ["Simple Avg (SVR + RF)", "Weighted Avg", "Stacked Ensemble (All 3)"]
errors = [error_simple, error_weighted, error_stacked]

for ax, name, error in zip(axs, titles, errors):
    ax.hist(error, bins=40, color='skyblue', edgecolor='black')
    ax.axvline(0, color='red', linestyle='--')
    ax.set_title(f"{name} Error Histogram", fontsize=14)
    ax.set_xlabel("Prediction Error (cm)", fontsize=14)
    ax.tick_params(axis='both', labelsize=14)

# Shared Y-axis label
axs[0].set_ylabel("Frequency", fontsize=14)

# Layout and optional save
plt.tight_layout()
plt.savefig("ensemble_error.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
true_stage = y_test.values
pred_simple = avg_svr_rf
pred_weighted = weighted_avg
pred_stacked = stacked_all_3

# Summary metrics
summary_metrics = pd.DataFrame({
    "Model": ["Simple Avg (2 models)", "Weighted Avg", "Stacked Ensemble (3 models)"],
    "R²": [
        r2_score(true_stage, pred_simple),
        r2_score(true_stage, pred_weighted),
        r2_score(true_stage, pred_stacked)
    ],
    "MAE": [
        mean_absolute_error(true_stage, pred_simple),
        mean_absolute_error(true_stage, pred_weighted),
        mean_absolute_error(true_stage, pred_stacked)
    ]
})

summary_metrics

In [ ]:
feature_importance_df = pd.DataFrame({
    "Feature": ["ROI1", "ROI2"],
    "Importance": [0.42, 0.58]
})

# Bar chart for feature importance
plt.figure(figsize=(6, 4))
plt.barh(feature_importance_df["Feature"], feature_importance_df["Importance"], color='skyblue')
plt.xlabel("Importance Score")
plt.title("Feature Importance (e.g., from Stacked Meta-Model)")
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Bland-Altman plot function
def bland_altman_plot(actual, predicted, model_name):
    mean_values = (actual + predicted) / 2
    diff = actual - predicted
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff

    plt.figure(figsize=(8, 5))
    plt.scatter(mean_values, diff, alpha=0.6, edgecolors='k', color='cornflowerblue')
    plt.axhline(mean_diff, color='red', linestyle='--', label=f'Mean Diff: {mean_diff:.3f}')
    plt.axhline(upper_limit, color='grey', linestyle='--', label=f'+1.96 SD: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='grey', linestyle='--', label=f'-1.96 SD: {lower_limit:.3f}')
    plt.fill_between(mean_values, lower_limit, upper_limit, color='grey', alpha=0.1)

    plt.title(f'Bland-Altman Plot ({model_name})', fontsize=14)
    plt.xlabel('Mean of Actual and Predicted Stage (cm)', fontsize=14)
    plt.ylabel('Residual (Actual - Predicted) (cm)', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True)
    plt.tick_params(axis='both', labelsize=14)
    plt.tight_layout()

    # Save figure
    plt.savefig(f"bland_altman_{model_name.lower().replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
    plt.show()

# Run for all three models
for model_name, y_pred in zip(
    ["Simple Avg (SVR + RF)", "Weighted Avg", "Stacked Ensemble (All 3)"],
    [avg_svr_rf, weighted_avg, stacked_all_3]
):
    bland_altman_plot(y_test, y_pred, model_name)

In [ ]:
# Prepare residuals
residuals = {
    "Simple Avg": true_stage - pred_simple,
    "Weighted Avg": true_stage - pred_weighted,
    "Stacked Ensemble": true_stage - pred_stacked
}

In [ ]:
# === Function to plot residuals over time ===
def plot_residuals_over_time(residuals_dict, timestamps):
    plt.figure(figsize=(14, 5))

    for label, resids in residuals_dict.items():
        plt.plot(timestamps, resids, label=label, alpha=0.7)

    plt.axhline(0, color='red', linestyle='--')
    plt.title("Residuals Over Time", fontsize=15)
    plt.xlabel("Timestamp", fontsize=13)
    plt.ylabel("Residual (Actual - Predicted) (cm)", fontsize=14)
    plt.legend(fontsize=11, loc="lower left")
    plt.grid(True)
    plt.tick_params(axis='both', labelsize=14)
    plt.tight_layout()

    # Save figure
    plt.savefig("ensemble_timeseries_residuals.png", dpi=300, bbox_inches='tight')
    plt.show()

# === Ensure timestamps are aligned with y_test ===
#timestamps_sorted = iqr_filtered_df.sort_values("image_timestamp").iloc[-len(y_test):]["image_timestamp"]
timestamps_sorted = iqr_filtered_df.loc[y_test.index, "image_timestamp"].sort_values()

# === Residuals dictionary structure ===
residuals = {
    "Simple Avg (SVR + RF)": avg_svr_rf - y_test,
    "Weighted Avg": weighted_avg - y_test,
    "Stacked Ensemble (All 3)": stacked_all_3 - y_test
}

# === Plot ===
plot_residuals_over_time(residuals_dict=residuals, timestamps=timestamps_sorted)


In [ ]:
# === Compute residuals ===
residual_simple = y_test - avg_svr_rf
residual_weighted = y_test - weighted_avg
residual_stacked = y_test - stacked_all_3

# === Create subplots ===
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# === Simple Avg plot ===
sns.kdeplot(
    residual_simple, ax=axes[0], fill=True,
    color="#1f77b4", alpha=0.5, linewidth=1
)
axes[0].axvline(x=0, color='black', linestyle='--', linewidth=1.2)
axes[0].set_title("Simple Avg (SVR + RF)", fontsize=14)
axes[0].set_xlabel("Prediction Error (cm)", fontsize=14)
axes[0].set_ylabel("Density", fontsize=14)
axes[0].grid(True, linestyle="--", alpha=0.3)

# === Weighted Avg plot ===
sns.kdeplot(
    residual_weighted, ax=axes[1], fill=True,
    color="#ff7f0e", alpha=0.4, linewidth=1
)
axes[1].axvline(x=0, color='black', linestyle='--', linewidth=1.2)
axes[1].set_title("Weighted Avg", fontsize=14)
axes[1].set_xlabel("Prediction Error (cm)", fontsize=14)
axes[1].grid(True, linestyle="--", alpha=0.3)

# === Stacked Ensemble plot ===
sns.kdeplot(
    residual_stacked, ax=axes[2], fill=True,
    color="#2ca02c", alpha=0.3, linewidth=1
)
axes[2].axvline(x=0, color='black', linestyle='--', linewidth=1.2)
axes[2].set_title("Stacked Ensemble (All 3)", fontsize=14)
axes[2].set_xlabel("Prediction Error (cm)", fontsize=14)
axes[2].grid(True, linestyle="--", alpha=0.3)

# === Final layout ===
plt.tight_layout()
plt.savefig("ensemble_error_pred_separate.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# === Compute residuals ===
residual_simple = y_test - avg_svr_rf
residual_weighted = y_test - weighted_avg
residual_stacked = y_test - stacked_all_3

# === Create KDE plot ===
plt.figure(figsize=(12, 6))

sns.kdeplot(
    residual_simple, label="Simple Avg (SVR + RF)", fill=True,
    color="#1f77b4", alpha=0.5, linewidth=1
)
sns.kdeplot(
    residual_weighted, label="Weighted Avg", fill=True,
    color="#ff7f0e", alpha=0.4, linewidth=1
)
sns.kdeplot(
    residual_stacked, label="Stacked Ensemble (All 3)", fill=True,
    color="#2ca02c", alpha=0.3, linewidth=1
)

# Reference line at zero
plt.axvline(x=0, color='black', linestyle='--', linewidth=1.2)

# === Labels and title ===
plt.title("Prediction Error Distribution (Ensemble Models)", fontsize=14)
plt.xlabel("Prediction Error (cm)", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.legend(fontsize=14)
plt.grid(True, linestyle="--", alpha=0.3)
plt.tick_params(axis='both', labelsize=14)
plt.tight_layout()

# Save figure
plt.savefig("ensemble_error_pred.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
from scipy.stats import skew, kurtosis

# Create a summary dataframe
summary_stats = pd.DataFrame({
    "Model": ["Simple Avg (2)", "Weighted Avg", "Stacked Ensemble (3)"],
    "Mean Error": [residual_simple.mean(), residual_weighted.mean(), residual_stacked.mean()],
    "Std Dev": [residual_simple.std(), residual_weighted.std(), residual_stacked.std()],
    "Skewness": [skew(residual_simple), skew(residual_weighted), skew(residual_stacked)],
    "Kurtosis": [kurtosis(residual_simple), kurtosis(residual_weighted), kurtosis(residual_stacked)]
})

summary_stats

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_prediction_band(timestamps, y_true, y_pred, model_name="Model", color="blue", band_cm=5):
    """
    Plot prediction with ±5 cm band
    """
    #band_ft = band_cm / 30.48  # Convert cm to feet
    band_ft = band_cm
    upper = y_pred + band_ft
    lower = y_pred - band_ft

    plt.figure(figsize=(14, 6))
    plt.plot(timestamps, y_true, label="True Stage", color="black", linewidth=2)
    plt.plot(timestamps, y_pred, label=f"Predicted ({model_name})", color=color, linewidth=1.5)
    plt.fill_between(timestamps, lower, upper, color=color, alpha=0.2, label=f"±{band_cm} cm Interval")
    plt.title(f"{model_name} Prediction with ±{band_cm} cm Band")
    plt.xlabel("Datetime")
    plt.ylabel("Stage (cm)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# Reconstruct test set with timestamps
X_test_with_time = X_test.copy()
X_test_with_time["timestamp"] = iqr_filtered_df.loc[X_test.index, "image_timestamp"]
X_test_with_time["true_stage"] = y_test.values
X_test_with_time["simple_avg"] = avg_svr_rf
X_test_with_time["weighted_avg"] = weighted_avg
X_test_with_time["stacked_ensemble"] = stacked_all_3

# Sort by timestamp
X_test_sorted = X_test_with_time.sort_values("timestamp")
timestamps_sorted = X_test_sorted["timestamp"]
y_test_true_sorted = X_test_sorted["true_stage"]
y_pred_simple_sorted = X_test_sorted["simple_avg"]
y_pred_weighted_sorted = X_test_sorted["weighted_avg"]
y_pred_stacked_sorted = X_test_sorted["stacked_ensemble"]


In [ ]:
plot_prediction_band(timestamps_sorted, y_test_true_sorted, y_pred_simple_sorted, model_name="Simple Avg (2 models)", color="blue")
plot_prediction_band(timestamps_sorted, y_test_true_sorted, y_pred_weighted_sorted, model_name="Weighted Avg", color="orange")
plot_prediction_band(timestamps_sorted, y_test_true_sorted, y_pred_stacked_sorted, model_name="Stacked Ensemble (3 models)", color="green")


# Stage Model Prediction

In [ ]:
import joblib

joblib.dump(svr_model, "svr_model.pkl")
joblib.dump(rf_tuned, "rf_model.pkl")
joblib.dump(xgb_model, "xgb_model.pkl")
meta_model_3 = LinearRegression().fit(stack_all_3, y_test)
joblib.dump(meta_model_3, "stacked_meta_model.pkl")


In [ ]:
import joblib
import numpy as np

# Load all models
svr_model = joblib.load("svr_model.pkl")
rf_model = joblib.load("rf_model.pkl")
xgb_model = joblib.load("xgb_model.pkl")
meta_model_3 = joblib.load("stacked_meta_model.pkl")

def predict_stage_from_segmentation(roi1, roi2):
    """
    Predict water stage from ROI1 and ROI2 (segmentation output)
    using stacked ensemble model.
    """
    features = np.array([[roi1, roi2]])

    # Base model predictions
    pred_svr = svr_model.predict(features)[0]
    pred_rf = rf_model.predict(features)[0]
    pred_xgb = xgb_model.predict(features)[0]

    # Stack and predict stage
    stacked_input = np.array([[pred_svr, pred_rf, pred_xgb]])
    stage_prediction = meta_model_3.predict(stacked_input)[0]

    return stage_prediction


In [ ]:
# Suppose your segmentation gives ROI1=0.62, ROI2=0.45
roi1 = 0.62
roi2 = 0.45

predicted_stage = predict_stage_from_segmentation(roi1, roi2)
print(f"Predicted Stage: {predicted_stage:.3f} ft")


In [ ]:
X_all = iqr_filtered_df[["ROI1", "ROI2"]].copy()
timestamps_all = iqr_filtered_df["image_timestamp"]

# Predict with base models
y_pred_svr_all = svr_model.predict(X_all)
y_pred_rf_all = rf_tuned.predict(X_all)
y_pred_xgb_all = xgb_model.predict(X_all)

# Stack predictions
stacked_input_all = np.vstack([y_pred_svr_all, y_pred_rf_all, y_pred_xgb_all]).T

# Predict stage using stacked ensemble model
stage_pred_all = meta_model_3.predict(stacked_input_all)

# Create output DataFrame
predicted_stage_df = pd.DataFrame({
    "timestamp": timestamps_all.values,
    "ROI1": X_all["ROI1"].values,
    "ROI2": X_all["ROI2"].values,
    "predicted_stage": stage_pred_all
})

predicted_stage_df

# Discharge

In [ ]:
from scipy.optimize import curve_fit
import pandas as pd

usgs_discharge_df = pd.read_csv("/content/USGS_Discharge_FD.csv", skiprows=1, header=None)

# Rename the needed columns based on previous structure
usgs_discharge_df = usgs_discharge_df.rename(columns={2: "datetime", 4: "discharge"})
usgs_discharge_df["datetime"] = pd.to_datetime(usgs_discharge_df["datetime"], errors="coerce")
usgs_discharge_df["discharge"] = pd.to_numeric(usgs_discharge_df["discharge"], errors="coerce")
usgs_discharge_df = usgs_discharge_df.dropna(subset=["datetime", "discharge"])
usgs_discharge_df["discharge"] *= 0.0283168
# Filter discharge data to only timestamps that exist in iqr_filtered_df_sorted
iqr_filtered_df_sorted["image_timestamp"] = pd.to_datetime(iqr_filtered_df_sorted["image_timestamp"])
timestamps_to_keep = iqr_filtered_df_sorted["image_timestamp"].unique()
filtered_discharge_df = usgs_discharge_df[usgs_discharge_df["datetime"].isin(timestamps_to_keep)]

# Merge stage and discharge for fitting
discharge_df = pd.merge(
    iqr_filtered_df_sorted,
    filtered_discharge_df[["datetime", "discharge"]],
    left_on="image_timestamp",
    right_on="datetime",
    how="inner"
).dropna(subset=["stage", "discharge"])

# Extract stage and discharge for curve fitting
h = discharge_df["stage"].values
Q = discharge_df["discharge"].values

# Define rating function and fit
def rating_curve(h, a, b, h0):
    return a * (h - h0) ** b

initial_guess = (1.0, 2.0, 1.5)
popt, _ = curve_fit(rating_curve, h, Q, p0=initial_guess)
a, b, h0 = popt

popt

In [ ]:
usgs_discharge_df.head()

In [ ]:
# Fitted rating curve parameters
a = 0.103
b = 1.177
h0 = 61.138

# Define discharge prediction function
def predict_discharge(h):
    h = np.array(h)
    return a * np.power(np.maximum(h - h0, 0), b)

# Apply to your in-memory DataFrame
predicted_stage_df["predicted_discharge_cms"] = predict_discharge(predicted_stage_df["predicted_stage"])
predicted_stage_df

In [ ]:
predicted_stage_df.to_csv("predicted_stage_discharge.csv", index=False)

In [ ]:
# Convert timestamp to datetime for both datasets
#predicted_stage_df["timestamp"] = pd.to_datetime(predicted_stage_df["timestamp"])
#filtered_discharge_df["datetime"] = pd.to_datetime(filtered_discharge_df["datetime"])

# Merge predicted discharge with observed discharge based on timestamp
comparison_df = pd.merge(
    predicted_stage_df,
    filtered_discharge_df[["datetime", "discharge"]],
    left_on="timestamp",
    right_on="datetime",
    how="inner"
).dropna(subset=["predicted_discharge_cms", "discharge"])

# Calculate performance metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

r2 = r2_score(comparison_df["discharge"], comparison_df["predicted_discharge_cms"])
mae = mean_absolute_error(comparison_df["discharge"], comparison_df["predicted_discharge_cms"])
mse = mean_squared_error(comparison_df["discharge"], comparison_df["predicted_discharge_cms"])

metrics = {
    "R²": r2,
    "MAE": mae,
    "MSE": mse
}

metrics

In [ ]:
import matplotlib.pyplot as plt

h_fit = np.linspace(h.min(), h.max(), 200)
Q_fit = rating_curve(h_fit, *popt)

plt.figure(figsize=(10, 6))
plt.scatter(h, Q, label="Observed Discharge", color="black", alpha=0.6)
plt.plot(h_fit, Q_fit, label="Fitted Rating Curve", color="red", linewidth=2)
plt.xlabel("Stage (cm)")
plt.ylabel("Discharge (cms)")
plt.title("Stage-Discharge Rating Curve")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("rating_curve.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Merge predicted discharge with observed discharge
discharge_comparison_df = pd.merge(
    predicted_stage_df[["timestamp", "predicted_discharge_cms"]],
    filtered_discharge_df[["datetime", "discharge"]],
    left_on="timestamp",
    right_on="datetime",
    how="inner"
).dropna()

# Sort by time for clean plotting
discharge_comparison_df = discharge_comparison_df.sort_values("timestamp")

# Plot
plt.figure(figsize=(14, 6))
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["discharge"], label="USGS Observed Discharge", linewidth=2)
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["predicted_discharge_cms"], label="Predicted Discharge", linestyle="--", linewidth=2)
plt.xlabel("Timestamp")
plt.ylabel("Discharge (cms)")
plt.title("Predicted vs Observed Discharge")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.signal import savgol_filter

# Apply Savitzky–Golay filter (window must be odd and >= polyorder + 2)
discharge_comparison_df["predicted_discharge_smoothed"] = savgol_filter(
    discharge_comparison_df["predicted_discharge_cms"], window_length=11, polyorder=2
)


In [ ]:
# Apply 5-point moving average (adjust window size as needed)
discharge_comparison_df["predicted_discharge_smoothed"] = (
    discharge_comparison_df["predicted_discharge_cfs"].rolling(window=5, center=True).mean()
)


In [ ]:
plt.figure(figsize=(14, 6))

# Plot observed and predicted discharge
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["discharge"],
         label="USGS Observed", linewidth=2)
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["predicted_discharge_smoothed"],
         label="Predicted", linestyle="--", linewidth=2)

# Set labels and title with larger fonts
plt.xlabel("Timestamp", fontsize=14)
plt.ylabel("Discharge (cms)", fontsize=14)
plt.title("Predicted vs Observed Discharge", fontsize=14)

# Tick font sizes
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Legend and grid
plt.legend(fontsize=14)
plt.grid(True)

# Layout
plt.tight_layout()
plt.savefig("discharge_timeseries.png", dpi=300, bbox_inches='tight')  # Optional save
plt.show()

In [ ]:
# Prepare values
x = comparison_df["discharge"].values.reshape(-1, 1)  # Observed
y = comparison_df["predicted_discharge_cms"].values  # Predicted

# Fit regression line
reg_model = LinearRegression()
reg_model.fit(x, y)
slope = reg_model.coef_[0]
intercept = reg_model.intercept_
y_pred_line = reg_model.predict(x)

# Regression equation string
reg_eq = f"y = {slope:.3f}x + {intercept:.3f}"

# Plot
plt.figure(figsize=(7, 7))
sns.scatterplot(x=comparison_df["discharge"], y=comparison_df["predicted_discharge_cms"],
                alpha=0.6, edgecolor='k', s=50, label="Predictions")
plt.plot([x.min(), x.max()], [x.min(), x.max()], 'r--', label="1:1 Line", linewidth=2)
plt.plot(x, y_pred_line, color="darkgreen", linestyle='-', linewidth=2, label=f"Regression Line\n({reg_eq})")

plt.xlabel("Observed Discharge (cms)", fontsize=14)
plt.ylabel("Predicted Discharge (cms)", fontsize=14)
plt.title("Observed vs Predicted Discharge with Regression Line", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True)
plt.tight_layout()
plt.savefig("discharge_scatter.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Compute residuals
comparison_df["residual"] = comparison_df["discharge"] - comparison_df["predicted_discharge_cfs"]

# Plot
plt.figure(figsize=(14, 6))
plt.plot(comparison_df["timestamp"], comparison_df["residual"], color="darkred", label="Discharge Residual")
plt.axhline(0, color='black', linestyle='--')
plt.xlabel("Timestamp")
plt.ylabel("Residual (cfs)")
plt.title("Discharge Prediction Error Over Time")
plt.grid(True)
plt.legend()
plt.tight_layout()
#plt.savefig("discharge_residuals_over_time.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 4))
sns.kdeplot(comparison_df["residual"], fill=True, color="steelblue", alpha=0.6)
plt.axvline(0, color='black', linestyle='--')
plt.xlabel("Prediction Error (cfs)")
plt.ylabel("Density")
plt.title("Distribution of Discharge Prediction Error")
plt.grid(True)
plt.tight_layout()
#plt.savefig("discharge_error_distribution.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
from scipy.optimize import curve_fit

#predicted_stage = pd.read_csv("/content/predicted_stage_discharge.csv", skiprows=1, header=None)
usgs_discharge_df = pd.read_csv("/content/USGS_Discharge_FD.csv", skiprows=1, header=None)

# Rename the needed columns based on previous structure
usgs_discharge_df = usgs_discharge_df.rename(columns={2: "datetime", 4: "discharge"})
usgs_discharge_df["datetime"] = pd.to_datetime(usgs_discharge_df["datetime"], errors="coerce")
usgs_discharge_df["discharge"] = pd.to_numeric(usgs_discharge_df["discharge"], errors="coerce")
usgs_discharge_df = usgs_discharge_df.dropna(subset=["datetime", "discharge"])

# Filter discharge data to only timestamps that exist in iqr_filtered_df_sorted
iqr_filtered_df_sorted["image_timestamp"] = pd.to_datetime(iqr_filtered_df_sorted["image_timestamp"])
timestamps_to_keep = iqr_filtered_df_sorted["image_timestamp"].unique()
filtered_discharge_df = usgs_discharge_df[usgs_discharge_df["datetime"].isin(timestamps_to_keep)]

# Merge stage and discharge for fitting
discharge_df = pd.merge(
    iqr_filtered_df_sorted,
    filtered_discharge_df[["datetime", "discharge"]],
    left_on="image_timestamp",
    right_on="datetime",
    how="inner"
).dropna(subset=["stage", "discharge"])

# Extract stage and discharge for curve fitting
h = discharge_df["stage"].values
Q = discharge_df["discharge"].values

# Define rating function and fit
def rating_curve(h, a, b, h0):
    return a * (h - h0) ** b

initial_guess = (1.0, 2.0, 1.5)
popt, _ = curve_fit(rating_curve, h, Q, p0=initial_guess)
a, b, h0 = popt

popt

In [ ]:
# Fitted rating curve parameters
a = 187.05
b = 1.313
h0 = 1.933

# Define discharge prediction function
def predict_discharge(h):
    h = np.array(h)
    return a * np.power(np.maximum(h - h0, 0), b)

# Apply to your in-memory DataFrame
predicted_stage_df["predicted_discharge_cfs"] = predict_discharge(predicted_stage_df["predicted_stage"])
predicted_stage_df

In [ ]:
predicted_stage_df.to_csv("predicted_stage_discharge.csv", index=False)

In [ ]:
# Convert timestamp to datetime for both datasets
#predicted_stage_df["timestamp"] = pd.to_datetime(predicted_stage_df["timestamp"])
#filtered_discharge_df["datetime"] = pd.to_datetime(filtered_discharge_df["datetime"])

# Merge predicted discharge with observed discharge based on timestamp
comparison_df = pd.merge(
    predicted_stage_df,
    filtered_discharge_df[["datetime", "discharge"]],
    left_on="timestamp",
    right_on="datetime",
    how="inner"
).dropna(subset=["predicted_discharge_cfs", "discharge"])

# Calculate performance metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

r2 = r2_score(comparison_df["discharge"], comparison_df["predicted_discharge_cfs"])
mae = mean_absolute_error(comparison_df["discharge"], comparison_df["predicted_discharge_cfs"])
mse = mean_squared_error(comparison_df["discharge"], comparison_df["predicted_discharge_cfs"])

metrics = {
    "R²": r2,
    "MAE": mae,
    "MSE": mse
}

metrics


In [ ]:
filtered_discharge_df

In [ ]:
# Merge predicted discharge with observed discharge
discharge_comparison_df = pd.merge(
    predicted_stage_df[["timestamp", "predicted_discharge_cfs"]],
    filtered_discharge_df[["datetime", "discharge"]],
    left_on="timestamp",
    right_on="datetime",
    how="inner"
).dropna()

# Sort by time for clean plotting
discharge_comparison_df = discharge_comparison_df.sort_values("timestamp")

# Plot
plt.figure(figsize=(14, 6))
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["discharge"], label="USGS Observed Discharge", linewidth=2)
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["predicted_discharge_cfs"], label="Predicted Discharge", linestyle="--", linewidth=2)
plt.xlabel("Timestamp")
plt.ylabel("Discharge (cfs)")
plt.title("Predicted vs Observed Discharge")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
discharge_comparison_df.to_csv("output_filename.csv", index=False)

In [ ]:
from scipy.signal import savgol_filter

# Apply Savitzky–Golay filter (window must be odd and >= polyorder + 2)
discharge_comparison_df["predicted_discharge_smoothed"] = savgol_filter(
    discharge_comparison_df["predicted_discharge_cfs"], window_length=11, polyorder=2
)


In [ ]:
# Apply 5-point moving average (adjust window size as needed)
discharge_comparison_df["predicted_discharge_smoothed"] = (
    discharge_comparison_df["predicted_discharge_cfs"].rolling(window=5, center=True).mean()
)


In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["discharge"], label="USGS Observed", linewidth=2)
plt.plot(discharge_comparison_df["timestamp"], discharge_comparison_df["predicted_discharge_smoothed"], label="Predicted", linestyle="--", linewidth=2)
plt.xlabel("Timestamp")
plt.ylabel("Discharge (cfs)")
plt.title("Predicted vs Observed Discharge")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
